<a href="https://colab.research.google.com/github/revan-desai/deep_learning/blob/main/bert_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/spam.csv')

In [5]:
df.sample(5)

,Category,Message
3019,ham,I didn't get the second half of that message
2249,ham,will you like to be spoiled? :)
1028,ham,Are you not around or just still asleep? :V
3016,ham,&lt;#&gt; mins but i had to stop somewhere fi...
4077,spam,87077: Kick off a new season with 2wks FREE go...


In [6]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [7]:
747/4825       # unbalanced data  15% are spam 85 % not spam

0.15481865284974095

In [8]:
df_spam=df[df['Category']=='spam']

df_spam.shape

df_spam.shape[0]

747

In [9]:
df_ham=df[df['Category']=='ham']

In [10]:
df_ham =df_ham.sample(df_spam.shape[0])

df_ham.shape

(747, 2)

In [102]:
df_balanced=pd.concat([df_ham,df_spam])

df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [103]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x :1 if x=='spam'  else 0 )

df_balanced.head(5)



,Category,Message,spam
1988,ham,No calls..messages..missed calls,0
2046,ham,Okay... I booked all already... Including the ...,0
3036,ham,Raji..pls do me a favour. Pls convey my Birthd...,0
4051,ham,Ok ok take care. I can understand.,0
3506,ham,"life alle mone,eppolum oru pole allalo",0


In [104]:
df_balanced=df_balanced.reset_index()

In [107]:
df_balanced=df_balanced.drop(['index'],axis=1)

In [108]:
df_balanced.head()

,Category,Message,spam
0,ham,No calls..messages..missed calls,0
1,ham,Okay... I booked all already... Including the ...,0
2,ham,Raji..pls do me a favour. Pls convey my Birthd...,0
3,ham,Ok ok take care. I can understand.,0
4,ham,"life alle mone,eppolum oru pole allalo",0


In [110]:
df_balanced.shape

(1494, 3)

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_test,y_train=train_test_split(df_balanced['Message'],df_balanced['spam'],stratify=df_balanced['spam'])  #



In [17]:
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_text as text

In [69]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [71]:
def get_sentence_embedding(sentences):
  preprocessed_text= bert_preprocess(sentences)
  bert_result=bert_encoder(preprocessed_text)['pooled_output']
  return bert_result

In [109]:
df_balanced['Message'][0]

'No calls..messages..missed calls'

In [114]:
list=[]

for i in range(10):
   text=get_sentence_embedding([df_balanced['Message'][i]])
   list.append(text)

In [115]:
list

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-0.9255782 , -0.59084845, -0.9761964 ,  0.86639166,  0.82462406,
         -0.34051386,  0.90713435,  0.3834606 , -0.94896394, -0.99999154,
         -0.7046508 ,  0.9715288 ,  0.98560673,  0.75685567,  0.92392385,
         -0.84004235, -0.68960565, -0.6600824 ,  0.4326062 , -0.34067035,
          0.78664243,  0.9999994 , -0.09074166,  0.41298714,  0.7479275 ,
          0.99795336, -0.85349864,  0.9061381 ,  0.9668049 ,  0.76088977,
         -0.7623649 ,  0.33540708, -0.9923255 , -0.3697512 , -0.98447436,
         -0.99333495,  0.45591742, -0.73926985, -0.23742509,  0.00254711,
         -0.8943545 ,  0.53742975,  0.9999933 ,  0.39756265,  0.52899516,
         -0.45078364, -1.        ,  0.4808164 , -0.907853  ,  0.97285587,
          0.9621749 ,  0.9720077 ,  0.32825795,  0.5876791 ,  0.67694795,
         -0.45904893,  0.21942817,  0.30718663, -0.2770039 , -0.7437607 ,
         -0.69954   ,  0.59766406, -0.95734566, -0.93702227, 

In [64]:
embedded_text=get_sentence_embedding(['billionaire','elon musk'])
embedded_text

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7521708 , -0.22597767,  0.48331374, ...,  0.4336694 ,
        -0.5467292 ,  0.7906744 ],
       [-0.7504134 , -0.2681261 , -0.26689705, ...,  0.02839382,
        -0.5938098 ,  0.7974989 ]], dtype=float32)>

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([embedded_text[0]],[embedded_text[1]])

array([[0.90725595]], dtype=float32)